 <center><h1>How I make a AI model to predict soccer game<h1></center>


![Drag Racing](./images/Soccer.png)

Here in Canada, it is the beginning of winter, going out becomes more and more difficult. People start to stay longer at home, they shorten their time spent outside. With this, it is the beginning of the world cup. 

And my bank account is empty. 💸

So I need a way to earn money without leaving my house. 

I quickly made the connection: I'm going to bet all my savings on the outcome of the World Cup.  But without taking any risk, in order to be sure of the result of the matches.

So I will try to build the perfect AI model to predict the results of the matches. 

## Step 1: Get the data

After long hours of research, I found this [website](https://www.football-data.co.uk/data.php) where I can find result of all the matchs from the 5 biggest European championship.

But this dataset alone will not be of much use to me. 

Among the most important parameters to create a coherent model, I need more information about the teams...

And for that, I thought that one of the most reliable sources would be the dataset of the game FIFA, which lists all the players of the big european championships, but also rates them, and gives me a lot of information that I can qualify as reliable and objective, even if they come from a video game

I found this dataset [here](https://www.kaggle.com/datasets/bryanb/fifa-player-stats-database?select=FIFA23_official_data.csv)



## Step 2: Clean the data

In [1]:
# This cell allow myself to have multiple outputs in one only cell 
# 
# https://www.roelpeters.be/jupyter-notebooks-how-to-print-multiple-outputs-in-a-cell/
from IPython.core.interactiveshell import InteractiveShell
import warnings
import json


In [2]:
# importantion de modules

import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import *
from sklearn.compose import *
import os
import glob
import difflib


In [3]:
pd.options.display.max_columns = 4000
InteractiveShell.ast_node_interactivity = "all"


In [4]:
# Constant
directory_PL_file = "./data/PremierLeague"


In [5]:
# merging the files
joined_files = os.path.join(directory_PL_file, "*.csv")

# A list of all joined files is returned
joined_list = glob.glob(joined_files)

# Finally, the files are joined
match = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

match.head()


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,Time,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,11/08/2017,Arsenal,Leicester,4,3,H,2,2,D,M Dean,27,6,10,3,9,12,9,4,0,1,0,0,1.53,4.5,6.50,1.50,4.60,6.75,1.47,4.5,6.50,1.44,4.40,6.50,1.53,4.55,6.85,1.53,4.2,6.00,1.53,4.50,6.50,41.0,1.55,1.51,4.60,4.43,6.89,6.44,37.0,1.65,1.61,2.43,2.32,21.0,-1.00,1.91,1.85,2.10,2.02,1.49,4.73,7.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,12/08/2017,Brighton,Man City,0,2,A,0,0,D,M Oliver,6,14,2,4,6,9,3,10,0,2,0,0,11.00,5.5,1.33,11.00,5.25,1.30,8.00,5.3,1.35,10.00,5.00,1.30,10.95,5.55,1.34,10.00,4.8,1.33,10.00,5.50,1.33,40.0,11.50,10.10,5.60,5.25,1.36,1.32,35.0,1.70,1.63,2.40,2.27,20.0,1.50,1.95,1.91,2.01,1.96,11.75,6.15,1.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,12/08/2017,Chelsea,Burnley,2,3,A,0,3,A,C Pawson,19,10,6,5,16,11,8,5,3,3,2,0,1.25,6.5,15.00,1.22,6.50,12.50,1.22,6.2,13.50,1.25,5.75,15.00,1.26,6.30,15.25,1.25,5.5,13.00,1.25,6.25,15.00,41.0,1.27,1.24,6.55,6.06,15.50,13.67,36.0,1.71,1.66,2.33,2.23,20.0,-1.75,2.03,1.97,1.95,1.90,1.33,5.40,12.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,12/08/2017,Crystal Palace,Huddersfield,0,3,A,0,2,A,J Moss,14,8,4,6,7,19,12,9,1,3,0,0,1.83,3.6,5.00,1.80,3.50,4.75,1.85,3.5,4.30,1.80,3.40,4.60,1.83,3.58,5.11,1.80,3.3,5.00,1.83,3.60,5.00,41.0,1.86,1.81,3.65,3.50,5.11,4.82,36.0,2.19,2.11,1.79,1.72,18.0,-0.75,2.10,2.05,1.86,1.83,1.79,3.56,5.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,12/08/2017,Everton,Stoke,1,0,H,1,0,H,N Swarbrick,9,9,4,1,13,10,6,7,1,1,0,0,1.70,3.8,5.75,1.70,3.60,5.50,1.70,3.7,5.00,1.67,3.60,5.25,1.70,3.83,5.81,1.70,3.5,5.50,1.70,3.80,5.75,40.0,1.71,1.69,3.85,3.69,6.00,5.50,35.0,2.17,2.08,1.80,1.76,19.0,-0.75,1.94,1.90,2.01,1.98,1.82,3.49,5.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
all_club_in_PL = match["HomeTeam"].unique()


In order to get all the club in Fifa, we need to get it from one dataset. We suppose that there is no club creation between years. In fact to be more precise, we assume that all the club we listed in the premier League are present in Fifa 22.

If a club is not in Premier league in Fifa 22 (but in the second division), there is no impact as we are just trying to make a connection beween dataset: we want to create a python dict that convert club from dataset match into club from fifa dataset

In [7]:
# Open Any Fifa dataset and get all the club into it
all_club_in_FIFA = pd.read_csv("./data/Fifa/FIFA22_official_data.csv")["Club"].astype(str).unique()


In [8]:
# Create the Dict
commonName = {}

# For each club in the dataset match, we try to see the most closest club string in the dataset
# from Fifa using difflib.get_close_matches 
# Source :
# https://stackoverflow.com/questions/10018679/python-find-closest-string-from-a-list-to-another-string

for club in all_club_in_PL:
    commonName[club] = difflib.get_close_matches(club, all_club_in_FIFA)

commonName

{'Arsenal': ['Arsenal', 'Arsenal Tula', 'Cesena'],
 'Brighton': [],
 'Chelsea': ['Chelsea', 'Cesena'],
 'Crystal Palace': ['Crystal Palace'],
 'Everton': ['Everton'],
 'Southampton': ['Southampton', 'Northampton Town'],
 'Watford': ['Watford', 'Waterford FC', 'Brentford'],
 'West Brom': ['West Bromwich Albion'],
 'Man United': ['Atlanta United', 'Manchester United', 'Sutton United'],
 'Newcastle': ['Newcastle Jets', 'Newcastle United'],
 'Bournemouth': ['AFC Bournemouth', 'Portsmouth'],
 'Burnley': ['Burnley', 'Barnsley', 'Bury'],
 'Leicester': ['Leicester City'],
 'Liverpool': ['Liverpool', 'Livorno', 'Liverpool Fútbol Club'],
 'Stoke': ['Stoke City'],
 'Swansea': ['Swansea City', 'Santos'],
 'Huddersfield': ['Huddersfield Town'],
 'Tottenham': ['Tottenham Hotspur'],
 'Man City': ['Swansea City', 'Manchester City', 'Mumbai City FC'],
 'West Ham': ['West Ham United'],
 'Fulham': ['Fulham'],
 'Leeds': ['Lecce'],
 'Aston Villa': ['Aston Villa'],
 'Wolves': [],
 'Brentford': ['Brentford',

As we see the result is not perfect: it cannot find any club for wolves, and sometimes, there is multiple choice. So I decided to go manually for the next step: create the dictionnary.
I just copy the results from the line above, and find what is corresponding in Fifa dataset.
What we have done cannot be entirely automatised, but it helps us a lot in order to make the dict quicker.

In [9]:
# Write the dict by hand, helped by the above code
PL_club = {
    "Arsenal": "Arsenal",
    "Brighton": "Brighton & Hove Albion",
    "Chelsea": "Chelsea",
    "Crystal Palace": "Crystal Palace",
    "Everton": "Everton",
    "Southampton": "Southampton",
    "Watford": "Watford",
    "West Brom": "West Bromwich Albion",
    "Man United": "Manchester United",
    "Newcastle": "Newcastle United",
    "Bournemouth": "AFC Bournemouth",
    "Burnley": "Burnley",
    "Leicester": "Leicester City",
    "Liverpool": "Liverpool",
    "Stoke": "Stoke City",
    "Swansea": "Swansea City",
    "Huddersfield": "Huddersfield Town",
    "Tottenham": "Tottenham Hotspur",
    "Man City": "Manchester City",
    "West Ham": "West Ham United",
    "Fulham": "Fulham",
    "Leeds": "Leeds United",
    "Aston Villa": "Aston Villa",
    "Wolves": "Wolverhampton Wanderers",
    "Brentford": "Brentford",
    "Nott'm Forest": "Nottingham Forest",
    "Cardiff": "Cardiff City",
    "Norwich": "Norwich City",
    "Sheffield United": "Sheffield United",
    "Hull": "Hull City",
    "Middlesbrough": "Middlesbrough",
    "Sunderland": "Sunderland",
}


In [10]:
# Save in file if we need to get the data later, or in another project
with open("./generated/club_pl.json", mode="w+") as outfile:
    json.dump(PL_club, outfile)